In [1]:
#!pip install vecstack
#!pip install bayesian-optimization
#!pip install catboost

In [2]:
%run import_modules.py

In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [3]:
X_train = pd.read_csv('train.csv').drop('index', axis=1)
y_train = X_train['voted']
X_train = X_train.drop('voted', axis=1)
X_test = pd.read_csv('test_x.csv').drop('index', axis=1)
train = pd.read_csv('train.csv')

In [4]:
### 변수정제
def to_int32(dataDF):
    Answers = dataDF.filter(regex='A').columns.tolist()
    
    ### float 데이터 -> int로 처리
    for feature in Answers:
        dataDF[feature] = dataDF[feature].astype('int32') 
    
    dataDF['Q_A_score'] = dataDF[Answers].mean(axis = 1)
    return dataDF

X_train = to_int32(X_train)
X_test = to_int32(X_test)


### 피처 추가
# 응답 종류 개수
X_train_Q_A = X_train.filter(like = 'A')
X_test_Q_A = X_test.filter(like = 'A')
X_train['Q_A_count'] = X_train_Q_A.nunique(axis = 1)
X_test['Q_A_count'] = X_test_Q_A.nunique(axis = 1)

# 응답평균시간
X_train_Q_E = X_train.filter(like = 'E')
X_test_Q_E = X_test.filter(like = 'E')
X_train['Q_E_mean'] = X_train_Q_E.mean(axis = 1)
X_test['Q_E_mean'] = X_test_Q_E.mean(axis = 1)

# 응답최대시간
X_train['Q_E_max'] = X_train_Q_E.max(axis = 1)
X_test['Q_E_max'] = X_test_Q_E.max(axis = 1)

# wr_(01-13) : 실존하는 해당 단어의 정의을 앎 & wf_(01-03) : 허구인 단어의 정의를 앎
# yes = 1, No = 0
# 합산을 통해서 얼마나 쌈박하게 똑똑한지
X_train_wr = X_train.filter(like = 'wr')
X_test_wr = X_test.filter(like = 'wr')
X_train_wf = X_train.filter(like = 'wf')
X_test_wf = X_test.filter(like = 'wf')
X_train['wr_sum'] = X_train_wr.sum(axis = 1)
X_test['wr_sum'] = X_test_wr.sum(axis = 1)
X_train['wf_sum'] = X_train_wf.sum(axis = 1)
X_test['wf_sum'] = X_test_wf.sum(axis = 1)

# 응답시간 변동계수
X_train['Q_E_cov'] = X_train_Q_E.std(axis = 1)/X_train_Q_E.mean(axis = 1)
X_test['Q_E_cov'] = X_test_Q_E.std(axis = 1)/X_test_Q_E.mean(axis = 1)

# 응답별 응답비율
X_train_total = X_train_Q_E.sum()
X_test_total = X_test_Q_E.sum()
X_train_Q_E_ratio = X_train_Q_E/X_train_total * 100000 # 단위가 너무 낮아서 100000 곱해줌
X_test_Q_E_ratio = X_test_Q_E/X_test_total * 100000
X_train_Q_E_ratio = X_train_Q_E_ratio.add_suffix('_ratio')
X_test_Q_E_ratio = X_test_Q_E_ratio.add_suffix('_ratio')
X_train = pd.concat([X_train, X_train_Q_E_ratio], axis = 1)
X_test = pd.concat([X_test, X_test_Q_E_ratio], axis = 1)



### 인코딩[라벨화]
le = LabelEncoder()
features = ['age_group', 'gender']
for feature in features:
    X_train[feature] = le.fit_transform(np.array(X_train[feature]))
    X_test[feature] = le.transform(np.array(X_test[feature]))

    
### 스케일링[RobustScaler]
robustScaler = RobustScaler()
features = X_train.filter(regex='E').columns.tolist()
X_train[features] = robustScaler.fit_transform(np.array(X_train[features])) 
X_test[features] = robustScaler.transform(np.array(X_test[features]))

### 원핫인코딩
def onehot_cat_features(dataDF):
    dataDF = pd.get_dummies(dataDF, columns = ['religion','race'])
    return dataDF
X_train = onehot_cat_features(X_train)
X_test = onehot_cat_features(X_test)

print(X_train.shape)
print(X_test.shape)

(45532, 120)
(11383, 120)


In [5]:
# cv 설정, 고정
skf = StratifiedKFold(n_splits=4 , shuffle=True, random_state=24)
sscv = StratifiedShuffleSplit(n_splits=4, random_state=24)
cv = skf

In [6]:
BO_tuned_clfs = []

<font color = 'blue'> 

# 첫번째 BO_tuned_clfs 

lgb_clf

In [7]:
pbounds = { 'learning_rate': (0.001, 0.06),
            'n_estimators': (90, 143),
            'max_depth': (6,15),   
            'min_child_samples' : (20, 30),
            'subsample': (0.8,0.92), 
            'colsample_bytree': (0.75,0.88),   
            'num_leaves': (25,45),
            'min_child_weight': (4, 10)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [8]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7687   |  0.8213   |  0.0432   |  11.42    |  25.45    |  6.542    |  124.2    |  33.75    |  0.907    |
|  2        |  0.7682   |  0.8753   |  0.02362  |  13.13    |  25.29    |  7.408    |  139.1    |  26.42    |  0.8105   |
|  3        |  0.7689   |  0.7526   |  0.05012  |  13.0     |  28.7     |  9.872    |  132.4    |  34.23    |  0.8937   |
|  4        |  0.7687   |  0.7654   |  0.03876  |  7.29     |  29.45    |  7.131    |  112.0    |  30.29    |  0.8929   |
|  5        |  0.7682   |  0.8093   |  0.03454  |  6.169    |  26.18    |  7.673    |  122.7    |  43.87    |  0.8818   |
|  6        |  0.7683   |  0.7967   |  0.02678  |  12.28    |  20.6     |  8.001    |  125.5    |  29.21    |  0.8155   |
|  7        |  0.7677   

|  41       |  0.7686   |  0.8299   |  0.04923  |  12.39    |  27.47    |  8.656    |  127.3    |  27.33    |  0.8401   |
|  42       |  0.7687   |  0.8496   |  0.03946  |  12.32    |  26.25    |  8.235    |  133.0    |  39.17    |  0.9014   |
|  43       |  0.7688   |  0.7974   |  0.03156  |  11.7     |  25.4     |  9.581    |  130.4    |  31.69    |  0.8185   |
|  44       |  0.7678   |  0.8288   |  0.02093  |  12.59    |  29.96    |  5.923    |  124.0    |  29.94    |  0.8372   |
|  45       |  0.7659   |  0.8264   |  0.008767 |  7.261    |  20.68    |  9.851    |  132.9    |  37.18    |  0.8105   |
|  46       |  0.7681   |  0.862    |  0.02667  |  9.587    |  25.79    |  7.603    |  110.8    |  32.0     |  0.9032   |
|  47       |  0.7677   |  0.8262   |  0.05591  |  10.7     |  23.04    |  9.31     |  100.4    |  38.65    |  0.8447   |
|  48       |  0.7685   |  0.862    |  0.04796  |  12.6     |  24.65    |  5.608    |  103.3    |  25.53    |  0.8118   |
|  49       |  0.768    

In [9]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.7823379086575946,
 'learning_rate': 0.03499328273065238,
 'max_depth': 11,
 'min_child_samples': 26,
 'min_child_weight': 5.33848979584371,
 'n_estimators': 140,
 'num_leaves': 34,
 'subsample': 0.9015690406965354,
 'objective': 'binary',
 'n_jobs': -1}

In [10]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.767445   0.76882148 0.77476588 0.76549308]
최대성능: 0.77476588321704
평균성능: 0.7691313613036108


<font color = 'blue'> 

# 두번째 BO_tuned_clfs 

lgb_clf

In [11]:
pbounds = { 'learning_rate': (0.001, 0.06),
            'n_estimators': (100, 160),
            'max_depth': (10,22),   
            'min_child_samples' : (20, 40),
            'subsample': (0.79 ,0.94), 
            'colsample_bytree': (0.76,0.9),   
            'num_leaves': (30,45),
            'min_child_weight': (5, 10)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [12]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7688   |  0.8368   |  0.0432   |  17.23    |  30.9     |  7.118    |  138.8    |  36.56    |  0.9238   |
|  2        |  0.7687   |  0.8949   |  0.02362  |  19.5     |  30.58    |  7.84     |  155.5    |  31.07    |  0.8031   |
|  3        |  0.7673   |  0.7628   |  0.05012  |  19.34    |  37.4     |  9.893    |  147.9    |  36.92    |  0.9071   |
|  4        |  0.7683   |  0.7766   |  0.03876  |  11.72    |  38.89    |  7.609    |  124.9    |  33.97    |  0.9061   |
|  5        |  0.7684   |  0.8239   |  0.03454  |  10.23    |  32.35    |  8.06     |  137.0    |  44.16    |  0.8923   |
|  6        |  0.7688   |  0.8103   |  0.02678  |  18.37    |  21.2     |  8.334    |  140.2    |  33.16    |  0.8093   |
|  7        |  0.7679   

|  41       |  0.7684   |  0.846    |  0.04923  |  18.51    |  34.94    |  8.88     |  142.2    |  31.75    |  0.8401   |
|  42       |  0.7683   |  0.8672   |  0.03946  |  18.43    |  32.5     |  8.529    |  148.7    |  40.63    |  0.9168   |
|  43       |  0.7685   |  0.811    |  0.03156  |  17.6     |  30.8     |  9.651    |  145.8    |  35.01    |  0.8131   |
|  44       |  0.7682   |  0.8448   |  0.02093  |  18.78    |  39.92    |  6.602    |  138.4    |  33.71    |  0.8366   |
|  45       |  0.7662   |  0.8423   |  0.008767 |  11.68    |  21.35    |  9.876    |  148.5    |  39.13    |  0.8031   |
|  46       |  0.7683   |  0.8806   |  0.02667  |  14.78    |  31.58    |  8.002    |  123.6    |  35.25    |  0.919    |
|  47       |  0.7683   |  0.842    |  0.05591  |  16.27    |  26.08    |  9.425    |  111.8    |  40.24    |  0.8458   |
|  48       |  0.7636   |  0.8706   |  0.001428 |  13.35    |  32.02    |  5.982    |  117.6    |  40.57    |  0.9247   |
|  49       |  0.7683   

In [13]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.8048024011151456,
 'learning_rate': 0.023624369756141978,
 'max_depth': 17,
 'min_child_samples': 37,
 'min_child_weight': 8.144909217955743,
 'n_estimators': 152,
 'num_leaves': 34,
 'subsample': 0.9097070250868845,
 'objective': 'binary',
 'n_jobs': -1}

In [14]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76731994 0.76820059 0.77431907 0.76574793]
최대성능: 0.7743190700256272
평균성능: 0.7688968809598804


<font color = 'blue'> 

# 세번째 BO_tuned_clfs 

lgb_clf

In [15]:
pbounds = { 'learning_rate': (0.001, 0.05),
            'n_estimators': (120, 190),
            'max_depth': (15, 30),   
            'min_child_samples' : (15, 40),
            'subsample': (0.78,0.93), 
            'colsample_bytree': (0.7,0.9),   
            'num_leaves': (15,40),
            'min_child_weight': (5, 15)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [16]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7686   |  0.8098   |  0.03604  |  24.04    |  28.62    |  9.237    |  165.2    |  25.94    |  0.9138   |
|  2        |  0.7673   |  0.8927   |  0.01979  |  26.88    |  28.22    |  10.68    |  184.8    |  16.78    |  0.7931   |
|  3        |  0.7688   |  0.704    |  0.0418   |  26.67    |  36.75    |  14.79    |  175.9    |  26.54    |  0.8971   |
|  4        |  0.7687   |  0.7237   |  0.03236  |  17.15    |  38.62    |  10.22    |  149.0    |  21.61    |  0.8961   |
|  5        |  0.7688   |  0.7912   |  0.02885  |  15.28    |  30.44    |  11.12    |  163.2    |  38.59    |  0.8823   |
|  6        |  0.7681   |  0.7719   |  0.02241  |  25.46    |  16.51    |  11.67    |  166.9    |  20.26    |  0.7993   |
|  7        |  0.7668   

|  41       |  0.7683   |  0.8229   |  0.04105  |  25.64    |  33.68    |  12.76    |  169.3    |  17.91    |  0.8301   |
|  42       |  0.7686   |  0.8532   |  0.03294  |  25.53    |  30.62    |  12.06    |  176.8    |  32.71    |  0.9068   |
|  43       |  0.7687   |  0.7729   |  0.02638  |  24.49    |  28.5     |  14.3     |  173.4    |  23.36    |  0.8031   |
|  44       |  0.7672   |  0.8212   |  0.01755  |  25.98    |  39.9     |  8.204    |  164.8    |  21.18    |  0.8266   |
|  45       |  0.7661   |  0.8175   |  0.007451 |  17.1     |  16.69    |  14.75    |  176.6    |  30.22    |  0.7931   |
|  46       |  0.7679   |  0.8723   |  0.02232  |  20.98    |  29.48    |  11.0     |  147.5    |  23.75    |  0.909    |
|  47       |  0.7681   |  0.8172   |  0.04661  |  22.84    |  22.6     |  13.85    |  133.8    |  32.07    |  0.8358   |
|  48       |  0.7639   |  0.858    |  0.001355 |  19.19    |  30.03    |  6.964    |  140.5    |  32.61    |  0.9147   |
|  49       |  0.7683   

In [17]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.8148650497699157,
 'learning_rate': 0.03300684017299955,
 'max_depth': 25,
 'min_child_samples': 26,
 'min_child_weight': 13.96546595851063,
 'n_estimators': 146,
 'num_leaves': 26,
 'subsample': 0.9137885032523508,
 'objective': 'binary',
 'n_jobs': -1}

In [18]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76685493 0.76844504 0.77417494 0.76595796]
최대성능: 0.774174938747321
평균성능: 0.7688582185923132


<font color = 'blue'> 

# 네번째 BO_tuned_clfs 

lgb_clf

In [19]:
pbounds = { 'learning_rate': (0.001, 0.05),
            'n_estimators': (140, 200),
            'max_depth': (15,30),   
            'min_child_samples' : (15, 30),
            'subsample': (0.79,0.92), 
            'colsample_bytree': (0.73,0.9),   
            'num_leaves': (15, 35),
            'min_child_weight': (5, 10)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [20]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7685   |  0.8233   |  0.03604  |  24.04    |  23.17    |  7.118    |  178.8    |  23.75    |  0.9059   |
|  2        |  0.7674   |  0.8938   |  0.01979  |  26.88    |  22.93    |  7.84     |  195.5    |  16.42    |  0.8013   |
|  3        |  0.7685   |  0.7334   |  0.0418   |  26.67    |  28.05    |  9.893    |  187.9    |  24.23    |  0.8915   |
|  4        |  0.7684   |  0.7501   |  0.03236  |  17.15    |  29.17    |  7.609    |  164.9    |  20.29    |  0.8907   |
|  5        |  0.7691   |  0.8075   |  0.02885  |  15.28    |  24.26    |  8.06     |  177.0    |  33.87    |  0.8786   |
|  6        |  0.7681   |  0.7911   |  0.02241  |  25.46    |  15.9     |  8.334    |  180.2    |  19.21    |  0.8068   |
|  7        |  0.7669   

|  41       |  0.7682   |  0.8344   |  0.04105  |  25.64    |  26.21    |  8.88     |  182.2    |  17.33    |  0.8334   |
|  42       |  0.7689   |  0.8602   |  0.03294  |  25.53    |  24.37    |  8.529    |  188.7    |  29.17    |  0.8999   |
|  43       |  0.7686   |  0.7919   |  0.02638  |  24.49    |  23.1     |  9.651    |  185.8    |  21.69    |  0.81     |
|  44       |  0.7674   |  0.833    |  0.01755  |  25.98    |  29.94    |  6.602    |  178.4    |  19.94    |  0.8303   |
|  45       |  0.7659   |  0.8299   |  0.007451 |  17.1     |  16.01    |  9.876    |  188.5    |  27.18    |  0.8013   |
|  46       |  0.7681   |  0.8764   |  0.02232  |  20.98    |  23.69    |  8.002    |  163.6    |  22.0     |  0.9018   |
|  47       |  0.7687   |  0.8296   |  0.04661  |  22.84    |  19.56    |  9.425    |  151.8    |  28.65    |  0.8384   |
|  48       |  0.7681   |  0.8764   |  0.04     |  26.01    |  21.97    |  6.34     |  155.0    |  15.53    |  0.8028   |
|  49       |  0.7683   

In [21]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.7849681205509292,
 'learning_rate': 0.02644550315385225,
 'max_depth': 22,
 'min_child_samples': 30,
 'min_child_weight': 9.156287297579574,
 'n_estimators': 184,
 'num_leaves': 31,
 'subsample': 0.8727605400071892,
 'objective': 'binary',
 'n_jobs': -1}

In [22]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76767758 0.76825724 0.77489874 0.7656943 ]
최대성능: 0.7748987406140267
평균성능: 0.7691319658456776


<font color = 'blue'> 

# 다섯번째 BO_tuned_clfs 

lgb_clf

In [23]:
pbounds = { 'learning_rate': (0.001, 0.05),
            'n_estimators': (150, 230),
            'max_depth': (15,30),   
            'min_child_samples' : (15, 35),
            'subsample': (0.8,0.95), 
            'colsample_bytree': (0.75,0.86),   
            'num_leaves': (20,40),
            'min_child_weight': (5, 15)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [24]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7687   |  0.8104   |  0.03604  |  24.04    |  25.9     |  9.237    |  201.7    |  28.75    |  0.9338   |
|  2        |  0.7683   |  0.856    |  0.01979  |  26.88    |  25.58    |  10.68    |  224.0    |  21.42    |  0.8131   |
|  3        |  0.7683   |  0.7522   |  0.0418   |  26.67    |  32.4     |  14.79    |  213.9    |  29.23    |  0.9171   |
|  4        |  0.7692   |  0.763    |  0.03236  |  17.15    |  33.89    |  10.22    |  183.2    |  25.29    |  0.9161   |
|  5        |  0.7689   |  0.8002   |  0.02885  |  15.28    |  27.35    |  11.12    |  199.4    |  38.87    |  0.9023   |
|  6        |  0.7687   |  0.7895   |  0.02241  |  25.46    |  16.2     |  11.67    |  203.7    |  24.21    |  0.8193   |
|  7        |  0.7676   

|  41       |  0.7685   |  0.8176   |  0.04105  |  25.64    |  29.94    |  12.76    |  206.3    |  22.33    |  0.8501   |
|  42       |  0.7688   |  0.8342   |  0.03294  |  25.53    |  27.5     |  12.06    |  214.9    |  34.17    |  0.9268   |
|  43       |  0.7688   |  0.7901   |  0.02638  |  24.49    |  25.8     |  14.3     |  211.0    |  26.69    |  0.8231   |
|  44       |  0.7683   |  0.8167   |  0.01755  |  25.98    |  34.92    |  8.204    |  201.3    |  24.94    |  0.8466   |
|  45       |  0.7667   |  0.8146   |  0.007451 |  17.1     |  16.35    |  14.75    |  214.7    |  32.18    |  0.8131   |
|  46       |  0.7686   |  0.8447   |  0.02232  |  20.98    |  26.58    |  11.0     |  181.4    |  27.0     |  0.929    |
|  47       |  0.7681   |  0.8145   |  0.04661  |  22.84    |  21.08    |  13.85    |  165.8    |  33.65    |  0.8558   |
|  48       |  0.7684   |  0.8447   |  0.04     |  26.01    |  24.3     |  7.68     |  170.1    |  20.53    |  0.8148   |
|  49       |  0.7678   

In [25]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.7678742228144012,
 'learning_rate': 0.031162418649908367,
 'max_depth': 17,
 'min_child_samples': 32,
 'min_child_weight': 13.073189587250107,
 'n_estimators': 196,
 'num_leaves': 28,
 'subsample': 0.8103750493182708,
 'objective': 'binary',
 'n_jobs': -1}

In [26]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76846999 0.76794561 0.77503107 0.76543603]
최대성능: 0.7750310685745978
평균성능: 0.7692206737930612


<font color = 'blue'> 

# 여섯번째 BO_tuned_clfs 

lgb_clf

In [27]:
pbounds = { 'learning_rate': (0.001, 0.05),
            'n_estimators': (100, 150),
            'max_depth': (10,20),   
            'min_child_samples' : (15, 30),
            'subsample': (0.8,0.92), 
            'colsample_bytree': (0.76,0.85),   
            'num_leaves': (15,35),
            'min_child_weight': (5, 10)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [28]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7686   |  0.8094   |  0.03604  |  16.03    |  23.17    |  7.118    |  132.3    |  23.75    |  0.907    |
|  2        |  0.7666   |  0.8467   |  0.01979  |  17.92    |  22.93    |  7.84     |  146.3    |  16.42    |  0.8105   |
|  3        |  0.7686   |  0.7618   |  0.0418   |  17.78    |  28.05    |  9.893    |  140.0    |  24.23    |  0.8937   |
|  4        |  0.7681   |  0.7706   |  0.03236  |  11.43    |  29.17    |  7.609    |  120.7    |  20.29    |  0.8929   |
|  5        |  0.7682   |  0.8011   |  0.02885  |  10.19    |  24.26    |  8.06     |  130.8    |  33.87    |  0.8818   |
|  6        |  0.7673   |  0.7924   |  0.02241  |  16.98    |  15.9     |  8.334    |  133.5    |  19.21    |  0.8155   |
|  7        |  0.766    

|  41       |  0.7684   |  0.8153   |  0.04105  |  17.1     |  26.21    |  8.88     |  135.2    |  17.33    |  0.8401   |
|  42       |  0.7686   |  0.8289   |  0.03294  |  17.02    |  24.37    |  8.529    |  140.6    |  29.17    |  0.9014   |
|  43       |  0.7681   |  0.7928   |  0.02638  |  16.33    |  23.1     |  9.651    |  138.1    |  21.69    |  0.8185   |
|  44       |  0.7687   |  0.7879   |  0.0448   |  17.66    |  27.29    |  8.723    |  137.0    |  22.19    |  0.8658   |
|  45       |  0.7688   |  0.8048   |  0.03769  |  15.6     |  26.35    |  8.58     |  137.6    |  26.51    |  0.8685   |
|  46       |  0.768    |  0.7735   |  0.05     |  16.72    |  28.83    |  9.231    |  139.4    |  29.28    |  0.8346   |
|  47       |  0.7682   |  0.8084   |  0.05     |  19.16    |  24.27    |  8.075    |  136.4    |  25.59    |  0.92     |
|  48       |  0.7676   |  0.8439   |  0.01786  |  14.31    |  25.97    |  6.99     |  141.8    |  26.34    |  0.8917   |
|  49       |  0.7683   

In [29]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.8048459143198399,
 'learning_rate': 0.03769440582215185,
 'max_depth': 16,
 'min_child_samples': 26,
 'min_child_weight': 8.579955953284205,
 'n_estimators': 138,
 'num_leaves': 27,
 'subsample': 0.8685450807553821,
 'objective': 'binary',
 'n_jobs': -1}

In [30]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76665318 0.76834005 0.77431269 0.76584877]
최대성능: 0.774312685645322
평균성능: 0.7687886730161093


<font color = 'blue'> 

# 일곱번째 BO_tuned_clfs 

lgb_clf

In [31]:
pbounds = { 'learning_rate': (0.001, 0.05),
            'n_estimators': (150, 200),
            'max_depth': (15,30),   
            'min_child_samples' : (20, 40),
            'subsample': (0.8,0.92), 
            'colsample_bytree': (0.76,0.9),   
            'num_leaves': (30,50),
            'min_child_weight': (6, 15)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [32]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7686   |  0.8368   |  0.03604  |  24.04    |  30.9     |  9.813    |  182.3    |  38.75    |  0.907    |
|  2        |  0.7687   |  0.8949   |  0.01979  |  26.88    |  30.58    |  11.11    |  196.3    |  31.42    |  0.8105   |
|  3        |  0.7683   |  0.7628   |  0.0418   |  26.67    |  37.4     |  14.81    |  190.0    |  39.23    |  0.8937   |
|  4        |  0.7693   |  0.7766   |  0.03236  |  17.15    |  38.89    |  10.7     |  170.7    |  35.29    |  0.8929   |
|  5        |  0.7686   |  0.8239   |  0.02885  |  15.28    |  32.35    |  11.51    |  180.8    |  48.87    |  0.8818   |
|  6        |  0.7689   |  0.8103   |  0.02241  |  25.46    |  21.2     |  12.0     |  183.5    |  34.21    |  0.8155   |
|  7        |  0.7683   

|  41       |  0.7682   |  0.8887   |  0.01903  |  20.62    |  37.59    |  13.45    |  166.5    |  35.57    |  0.8148   |
|  42       |  0.7688   |  0.8672   |  0.03294  |  25.53    |  32.5     |  12.35    |  190.6    |  44.17    |  0.9014   |
|  43       |  0.769    |  0.811    |  0.02638  |  24.49    |  30.8     |  14.37    |  188.1    |  36.69    |  0.8185   |
|  44       |  0.7684   |  0.8448   |  0.01755  |  25.98    |  39.92    |  8.884    |  182.0    |  34.94    |  0.8372   |
|  45       |  0.7662   |  0.8423   |  0.007451 |  17.1     |  21.35    |  14.78    |  190.4    |  42.18    |  0.8105   |
|  46       |  0.7687   |  0.8806   |  0.02232  |  20.98    |  31.58    |  11.4     |  169.6    |  37.0     |  0.9032   |
|  47       |  0.7676   |  0.842    |  0.04661  |  22.84    |  26.08    |  13.97    |  159.9    |  43.65    |  0.8447   |
|  48       |  0.7686   |  0.8806   |  0.04     |  26.01    |  29.3     |  8.412    |  162.5    |  30.53    |  0.8118   |
|  49       |  0.7694   

In [33]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.7604296531524981,
 'learning_rate': 0.02307705418401338,
 'max_depth': 26,
 'min_child_samples': 34,
 'min_child_weight': 10.666818829880135,
 'n_estimators': 169,
 'num_leaves': 39,
 'subsample': 0.8577600405357944,
 'objective': 'binary',
 'n_jobs': -1}

In [34]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76819336 0.76845255 0.77475984 0.76625691]
최대성능: 0.7747598414132391
평균성능: 0.7694156640221028


<font color = 'blue'> 

# 여덟번째 BO_tuned_clfs 

lgb_clf

In [35]:
pbounds = { 'learning_rate': (0.001, 0.05),
            'n_estimators': (200, 250),
            'max_depth': (20,35),   
            'min_child_samples' : (25, 40),
            'subsample': (0.8,0.92), 
            'colsample_bytree': (0.76,0.87),   
            'num_leaves': (25,50),
            'min_child_weight': (5, 15)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [36]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7684   |  0.8204   |  0.03604  |  29.04    |  33.17    |  9.237    |  232.3    |  35.94    |  0.907    |
|  2        |  0.7693   |  0.866    |  0.01979  |  31.88    |  32.93    |  10.68    |  246.3    |  26.78    |  0.8105   |
|  3        |  0.7676   |  0.7622   |  0.0418   |  31.67    |  38.05    |  14.79    |  240.0    |  36.54    |  0.8937   |
|  4        |  0.7685   |  0.773    |  0.03236  |  22.15    |  39.17    |  10.22    |  220.7    |  31.61    |  0.8929   |
|  5        |  0.7684   |  0.8102   |  0.02885  |  20.28    |  34.26    |  11.12    |  230.8    |  48.59    |  0.8818   |
|  6        |  0.769    |  0.7995   |  0.02241  |  30.46    |  25.9     |  11.67    |  233.5    |  30.26    |  0.8155   |
|  7        |  0.7683   

|  41       |  0.7688   |  0.8483   |  0.03102  |  29.61    |  36.12    |  10.89    |  246.1    |  27.76    |  0.8095   |
|  42       |  0.7685   |  0.8568   |  0.01497  |  32.32    |  33.79    |  9.075    |  242.8    |  28.53    |  0.8162   |
|  43       |  0.7687   |  0.865    |  0.03621  |  25.83    |  35.35    |  9.586    |  211.7    |  37.38    |  0.9005   |
|  44       |  0.7659   |  0.797    |  0.005701 |  33.82    |  33.26    |  14.22    |  248.5    |  26.17    |  0.8354   |
|  45       |  0.7689   |  0.8632   |  0.02482  |  30.7     |  33.7     |  9.209    |  245.7    |  27.34    |  0.8071   |
|  46       |  0.7693   |  0.8187   |  0.02651  |  24.04    |  34.61    |  11.78    |  214.3    |  36.96    |  0.81     |
|  47       |  0.7684   |  0.8442   |  0.03692  |  22.66    |  34.03    |  10.3     |  213.1    |  34.39    |  0.8472   |
|  48       |  0.7689   |  0.7631   |  0.02377  |  23.29    |  36.94    |  11.45    |  216.4    |  38.51    |  0.8094   |
|  49       |  0.765    

In [37]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.8660029036551132,
 'learning_rate': 0.01978863442246311,
 'max_depth': 32,
 'min_child_samples': 33,
 'min_child_weight': 10.680445610939323,
 'n_estimators': 246,
 'num_leaves': 27,
 'subsample': 0.810455515964185,
 'objective': 'binary',
 'n_jobs': -1}

In [38]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76782907 0.76869036 0.7746348  0.7661447 ]
최대성능: 0.7746348009892114
평균성능: 0.7693247322680086


<font color = 'blue'> 

# 아홉번째 BO_tuned_clfs 

lgb_clf

In [39]:
pbounds = { 'learning_rate': (0.001, 0.05),
            'n_estimators': (100, 200),
            'max_depth': (15,25),   
            'min_child_samples' : (20, 35),
            'subsample': (0.8,0.92), 
            'colsample_bytree': (0.75,0.85),   
            'num_leaves': (20,40),
            'min_child_weight': (5, 15)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [40]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7687   |  0.8049   |  0.03604  |  21.03    |  28.17    |  9.237    |  164.6    |  28.75    |  0.907    |
|  2        |  0.768    |  0.8464   |  0.01979  |  22.92    |  27.93    |  10.68    |  192.6    |  21.42    |  0.8105   |
|  3        |  0.7685   |  0.752    |  0.0418   |  22.78    |  33.05    |  14.79    |  179.9    |  29.23    |  0.8937   |
|  4        |  0.7684   |  0.7618   |  0.03236  |  16.43    |  34.17    |  10.22    |  141.5    |  25.29    |  0.8929   |
|  5        |  0.7688   |  0.7956   |  0.02885  |  15.19    |  29.26    |  11.12    |  161.7    |  38.87    |  0.8818   |
|  6        |  0.7684   |  0.786    |  0.02241  |  21.98    |  20.9     |  11.67    |  167.1    |  24.21    |  0.8155   |
|  7        |  0.7668   

|  41       |  0.7676   |  0.7796   |  0.04642  |  21.15    |  28.75    |  8.658    |  195.7    |  28.61    |  0.8761   |
|  42       |  0.769    |  0.8266   |  0.03294  |  22.02    |  29.37    |  12.06    |  181.1    |  34.17    |  0.9014   |
|  43       |  0.7687   |  0.7864   |  0.02638  |  21.33    |  28.1     |  14.3     |  176.3    |  26.69    |  0.8185   |
|  44       |  0.768    |  0.8106   |  0.01755  |  22.32    |  34.94    |  8.204    |  164.1    |  24.94    |  0.8372   |
|  45       |  0.7663   |  0.8088   |  0.007451 |  16.4     |  21.01    |  14.75    |  180.9    |  32.18    |  0.8105   |
|  46       |  0.768    |  0.8361   |  0.02232  |  18.99    |  28.69    |  11.0     |  139.3    |  27.0     |  0.9032   |
|  47       |  0.7685   |  0.8086   |  0.04661  |  20.22    |  24.56    |  13.85    |  119.7    |  33.65    |  0.8447   |
|  48       |  0.7683   |  0.8361   |  0.04     |  22.34    |  26.97    |  7.68     |  125.1    |  20.53    |  0.8118   |
|  49       |  0.7681   

In [41]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.7644847759343377,
 'learning_rate': 0.024914757751798775,
 'max_depth': 19,
 'min_child_samples': 34,
 'min_child_weight': 12.653252538069653,
 'n_estimators': 175,
 'num_leaves': 38,
 'subsample': 0.8100106922530422,
 'objective': 'binary',
 'n_jobs': -1}

In [42]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76795181 0.76795324 0.7748169  0.76530112]
최대성능: 0.774816895972845
평균성능: 0.7690057650859187


<font color = 'blue'> 

# 열번째 BO_tuned_clfs 

lgb_clf

In [43]:
pbounds = { 'learning_rate': (0.001, 0.1),
            'n_estimators': (170, 230),
            'max_depth': (15,25),   
            'min_child_samples' : (20, 35),
            'subsample': (0.8,0.92), 
            'colsample_bytree': (0.76,0.87),   
            'num_leaves': (30,45),
            'min_child_weight': (5, 15)}


def lgbm_opt(learning_rate, n_estimators, max_depth, min_child_samples, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'min_child_samples' : int(round(min_child_samples)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'objective' : 'binary',
        'n_jobs' : -1
    }
    
    lgbm = LGBMClassifier(**params)
    
    score = cross_val_score(lgbm, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    return np.mean(score)

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=0)    


In [44]:
BO_lgbm.maximize(init_points=40, n_iter=10)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  0.7645   |  0.8204   |  0.0718   |  21.03    |  28.17    |  9.237    |  208.8    |  36.56    |  0.907    |
|  2        |  0.7684   |  0.866    |  0.03896  |  22.92    |  27.93    |  10.68    |  225.5    |  31.07    |  0.8105   |
|  3        |  0.7627   |  0.7622   |  0.08343  |  22.78    |  33.05    |  14.79    |  217.9    |  36.92    |  0.8937   |
|  4        |  0.7655   |  0.773    |  0.06435  |  16.43    |  34.17    |  10.22    |  194.9    |  33.97    |  0.8929   |
|  5        |  0.7656   |  0.8102   |  0.05727  |  15.19    |  29.26    |  11.12    |  207.0    |  44.16    |  0.8818   |
|  6        |  0.7677   |  0.7995   |  0.04427  |  21.98    |  20.9     |  11.67    |  210.2    |  33.16    |  0.8155   |
|  7        |  0.7685   

|  41       |  0.7649   |  0.7913   |  0.07469  |  17.43    |  21.08    |  9.504    |  171.1    |  41.04    |  0.8298   |
|  42       |  0.7653   |  0.8442   |  0.06554  |  22.02    |  29.37    |  12.06    |  218.7    |  40.63    |  0.9014   |
|  43       |  0.7668   |  0.8001   |  0.05228  |  21.33    |  28.1     |  14.3     |  215.8    |  35.01    |  0.8185   |
|  44       |  0.7691   |  0.8267   |  0.03444  |  22.32    |  34.94    |  8.204    |  208.4    |  33.71    |  0.8372   |
|  45       |  0.7684   |  0.8246   |  0.01403  |  16.4     |  21.01    |  14.75    |  218.5    |  39.13    |  0.8105   |
|  46       |  0.7683   |  0.8547   |  0.04408  |  18.99    |  28.69    |  11.0     |  193.6    |  35.25    |  0.9032   |
|  47       |  0.7625   |  0.8245   |  0.09314  |  20.22    |  24.56    |  13.85    |  181.8    |  40.24    |  0.8447   |
|  48       |  0.7641   |  0.7733   |  0.08257  |  16.23    |  21.47    |  12.7     |  170.6    |  41.15    |  0.8171   |
|  49       |  0.7632   

In [45]:
# BO_rf.res  # 모든 성능 들어가있음
BO_lgbm.max

max_params = BO_lgbm.max['params']

max_params['n_estimators'] = int(round(max_params['n_estimators']))
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['objective'] = 'binary'
max_params['n_jobs'] = -1

max_params

{'colsample_bytree': 0.8627252797918619,
 'learning_rate': 0.024977784627540384,
 'max_depth': 18,
 'min_child_samples': 30,
 'min_child_weight': 8.164973432185569,
 'n_estimators': 220,
 'num_leaves': 33,
 'subsample': 0.8216326752419001,
 'objective': 'binary',
 'n_jobs': -1}

In [46]:
lgbm_clf = LGBMClassifier(**max_params)

scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

print(scores)
print(f'최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')

BO_tuned_clfs.append((lgbm_clf.__class__.__name__, lgbm_clf, np.mean(scores)))

[0.76757169 0.76830287 0.77472642 0.76678731]
최대성능: 0.7747264246324217
평균성능: 0.7693470722468116


<font color = 'blue'> 

# best 모델 반환

In [47]:
for tuned_clf in BO_tuned_clfs:
    
    print(f'{tuned_clf}\n\n\n')

('LGBMClassifier', LGBMClassifier(colsample_bytree=0.7823379086575946,
               learning_rate=0.03499328273065238, max_depth=11,
               min_child_samples=26, min_child_weight=5.33848979584371,
               n_estimators=140, num_leaves=34, objective='binary',
               subsample=0.9015690406965354), 0.7691313613036108)



('LGBMClassifier', LGBMClassifier(colsample_bytree=0.8048024011151456,
               learning_rate=0.023624369756141978, max_depth=17,
               min_child_samples=37, min_child_weight=8.144909217955743,
               n_estimators=152, num_leaves=34, objective='binary',
               subsample=0.9097070250868845), 0.7688968809598804)



('LGBMClassifier', LGBMClassifier(colsample_bytree=0.8148650497699157,
               learning_rate=0.03300684017299955, max_depth=25,
               min_child_samples=26, min_child_weight=13.96546595851063,
               n_estimators=146, num_leaves=26, objective='binary',
               subsample=0.9137885

In [48]:
max_score = 0
for tuned_clf in BO_tuned_clfs:
    if tuned_clf[2] > max_score:
        max_score = tuned_clf[2]

    # best 점수를 가진 모델 출력
for tuned_clf in BO_tuned_clfs:
    if tuned_clf[2] == max_score:
        print(f'{tuned_clf}')
    # 사용된 cv 출력
print(f'\n\n{cv}')

('LGBMClassifier', LGBMClassifier(colsample_bytree=0.7604296531524981,
               learning_rate=0.02307705418401338, max_depth=26,
               min_child_samples=34, min_child_weight=10.666818829880135,
               n_estimators=169, num_leaves=39, objective='binary',
               subsample=0.8577600405357944), 0.7694156640221028)


StratifiedKFold(n_splits=4, random_state=24, shuffle=True)
